In [ ]:
import numpy as np

import pandas as pd
import pandas_ta as ta # pandas technical analysis
import pandas_datareader.data as web

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,6)
plt.style.use("classic") # plt.style.available[:] gives list of all available stylesheets
plt.rcParams["font.size"] = 11
plt.rcParams["lines.linestyle"] = "-"
plt.rcParams["figure.dpi"] = 700 #sharp image

import seaborn as sns

from datetime import datetime

import yfinance as yf

from ta.momentum import RSIIndicator
from ta.momentum import ROCIndicator
from ta.momentum import StochasticOscillator
from ta.momentum import WilliamsRIndicator

from ta.trend import MACD
from ta.trend import EMAIndicator
from ta.trend import WMAIndicator
from ta.trend import SMAIndicator

from ta.volatility import BollingerBands

from ta.volume import OnBalanceVolumeIndicator

In [ ]:
end = datetime.today()
start = datetime(end.year-1, end.month, end.day)

ticker = ["^GSPC", 
          "^IXIC"
         ] # add tickers here

df = web.DataReader(ticker, "yahoo", start, end)

In [ ]:
# .stack() converts the innermost column index into the innermost row index
df = df.stack() # https://medium.com/swlh/reshaping-in-pandas-with-stack-and-unstack-functions-bb169f64467d
df = df.reset_index() # not necessary, puts the same date to each innermost row index
df

In [ ]:
class TechnicalIndicators:
    """
    This is an example of the TechIndicator class.
    """
    def __init__(self, symbol, close, high, low, volume):
        self.symbol = symbol
        self.close = close
        self.high = high
        self.low = low
        self.volume = volume
        self.technical_indicators = None
        
    def set_technical_indicators(self):
        self.technical_indicators = pd.DataFrame()
        
        self.technical_indicators["Close"] = self.close
        self.technical_indicators["High"] = self.high
        self.technical_indicators["Low"] = self.low
        self.technical_indicators["Volume"] = self.volume

# momentum indicators
    def rate_change(self, window):
        """
        Rate of Change. Momentum Indicator
        """
        closing_price = self.close
        dataframe = self.technical_indicators
        
        roc = ROCIndicator(
                    close = closing_price,
                    window = window,
                    fillna = False
                    )
        
        dataframe["ROC"] = roc.roc()

        return dataframe    
    
    def rsi(self, window):
        """
        The relative strength index. Momentum Indicator.
        """
        closing_price = self.close
        dataframe = self.technical_indicators
        
        rsi = RSIIndicator(
                        close = closing_price, 
                        window = window,
                        fillna = False
                          )
        
        dataframe["RSI"] = rsi.rsi()
        
        return dataframe
    
    def stochastic_oscillator(self, window):
        """
        Stochastic Oscillator. Momentum Indicator.
        """
        dataframe = self.technical_indicators
        
        stoch_osci = StochasticOscillator(
            close = self.close,
            high = self.high,
            low = self.low,
            window = window,
            smooth_window = 3,
            fillna = False
             )
        
        dataframe["Stochastic Oscillator"] = stoch_osci.stoch()
    
        return dataframe
    
    def WilliamsR(self, window):
        """
        WilliamsR Indicator. Momentum Indicator.
        """
        dataframe = self.technical_indicators

        williamsR = WilliamsRIndicator(
                                    high = self.high,
                                    low = self.low,
                                    close = self.close,
                                    lbp = window,
                                    fillna = False
                                      )
    
        dataframe["Williams %R Indicator"] = williamsR.williams_r()
    
        return dataframe
    
# trend indicators
    def moving_average(self):
        """
        Moving Average.
        """
        dataframe = self.technical_indicators
        days = [5,10,20]
        
        for i in days:
            col_name = f"MA for {i} days"
            dataframe[col_name] = self.price.rolling(window = i).mean()
            
        return dataframe
    
    def macd(self):
        """
        The moving average convergence/divergence.
        """
        dataframe = self.technical_indicators
        closing_price = self.close
        ema_26 = 26
        ema_12 = 12
        signal = 9
        macd = MACD(
            closing_price, 
            window_fast = ema_12, 
            window_slow = ema_26,
            window_sign = signal,
            fillna = False
                    )
        dataframe["MACD"] = macd.macd()
        
        return dataframe

    def ema(self, window):
        """
        The exponential moving average. Trend indicator.
        """
        dataframe = self.technical_indicators
    
        ema = EMAIndicator(
                close = self.close,
                window = window,
                fillna = False
                    )
        
        dataframe["Exponential moving average"] = ema.ema_indicator()
        
        return dataframe
    
    def wma(self, window):
        """
        Weighted moving average. Trend indicator.
        """
        dataframe = self.technical_indicators
        
        wma = WMAIndicator(
                close = self.close, 
                window = window, 
                fillna = False
                )
        
        dataframe["Weighted moving average"] = wma.wma()
        
        return dataframe

# volatility indicators
    def bollinger_bands(self, window):
        """
        Bollinger bands. Volatility Indicator.
        """
        closing_price = self.close
        dataframe = self.technical_indicators
        
        bbands = BollingerBands(
                            close = closing_price,
                            window = window,
                            window_dev = 2,
                            fillna= False
                               )

        dataframe["Bollinger Bands Middle"] = bbands.bollinger_mavg()
        dataframe["Bollinger Bands Lower"] = bbands.bollinger_hband()
        dataframe["Bollinger Bands Upper"] = bbands.bollinger_lband()
        
        return dataframe
    
# volume indicators
    def on_balance_volume(self):
        """
        On Balance Volume. Volume Indicator.
        """
        dataframe = self.technical_indicators
        
        on_bal_vol = OnBalanceVolumeIndicator(
                                close = self.close,
                                volume = self.volume,
                                fillna = False
                                  )
        
        dataframe["On Balance Volume"] = on_bal_vol.on_balance_volume()
        
        return dataframe

In [ ]:
gspc = df[df["Symbols"] == "^GSPC"]
ixic = df[df["Symbols"] == "^IXIC"]

symbol = TechnicalIndicators(gspc, gspc["Close"], gspc["High"], gspc["Low"], gspc["Volume"])
symbol.set_technical_indicators()
symbol.rsi(14)
symbol.bollinger_bands(20)
symbol.rate_change(30).tail(40)
symbol.rate_change(1)
symbol.stochastic_oscillator(14)
symbol.WilliamsR(14)
symbol.on_balance_volume()
symbol.ema(14)
symbol.wma(14)